In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
movie_df = pd.read_csv("/content/drive/MyDrive/Azure/movie_titles.csv", encoding="latin1", usecols=[0, 1, 2], header = None)
movie_df.columns = ["movie_id", "year", "name"]
movie_df.info()
print(movie_df.head())
print("===========Null nums===========")
print(movie_df.isnull().sum())
print("===========Duplicated nums===========")
print(movie_df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17770 entries, 0 to 17769
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  17770 non-null  int64  
 1   year      17763 non-null  float64
 2   name      17770 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 416.6+ KB
   movie_id    year                          name
0         1  2003.0               Dinosaur Planet
1         2  2004.0    Isle of Man TT 2004 Review
2         3  1997.0                     Character
3         4  1994.0  Paula Abdul's Get Up & Dance
4         5  2004.0      The Rise and Fall of ECW
===========Null nums===========
movie_id    0
year        7
name        0
dtype: int64
===========Duplicated nums===========
0


In [ ]:
interact_df_1 = pd.read_csv("/content/drive/MyDrive/Azure/combined_data_1.txt", skiprows = 1, sep = ',', usecols=[0, 1, 2], header = None)
interact_df_2 = pd.read_csv("/content/drive/MyDrive/Azure/combined_data_2.txt", skiprows = 1, sep = ',', usecols=[0, 1, 2], header = None)
interact_df = pd.concat([interact_df_1, interact_df_2])
interact_df.columns = ["user_id", "movie_id", "date"]
interact_df.info()
print(interact_df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 51040563 entries, 0 to 26982300
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   object 
 1   movie_id  float64
 2   date      object 
dtypes: float64(1), object(2)
memory usage: 1.5+ GB
   user_id  movie_id        date
0  1488844       3.0  2005-09-06
1   822109       5.0  2005-05-13
2   885013       4.0  2005-10-19
3    30878       4.0  2005-12-26
4   823519       3.0  2004-05-03


In [ ]:
interact_df.isnull().sum()

,0
user_id,0
movie_id,9208
date,9208


In [ ]:
interact_df = interact_df[interact_df["movie_id"].notna()]

In [ ]:
duplicated_df = interact_df[interact_df.duplicated()]
print(duplicated_df)

          user_id  movie_id        date
689        515436       1.0  2005-02-13
3041       636262       1.0  2005-08-23
3221       618272       1.0  2005-08-14
3435       261764       1.0  2005-11-10
3607      2147527       1.0  2005-11-18
...           ...       ...         ...
26982296  2420260       1.0  2003-12-01
26982297   761176       3.0  2004-06-06
26982298   459277       3.0  2005-02-25
26982299  2407365       4.0  2005-04-29
26982300   627867       3.0  2005-05-02

[31178906 rows x 3 columns]


In [ ]:
interact_df = interact_df.drop_duplicates()

In [ ]:
interact_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19852449 entries, 0 to 26982295
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   object 
 1   movie_id  float64
 2   date      object 
dtypes: float64(1), object(2)
memory usage: 605.8+ MB


In [ ]:
interact_df['date'] = pd.to_datetime(interact_df['date'])

user_movies = (
    interact_df.sort_values(["user_id", "date"], ascending=[True, False])
        .groupby("user_id")
        .head(20)
)

/tmp/ipython-input-3277328941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interact_df['date'] = pd.to_datetime(interact_df['date'])


In [ ]:
user_movies.to_parquet("user_movies.parquet", engine="pyarrow", index=False)
movie_df.to_parquet("movie_df.parquet", engine="pyarrow", index=False)

In [ ]:
import pandas as pd
import random

user_id = user_movies["user_id"].unique()

ho = ["Nguyễn", "Trần", "Lê", "Phạm", "Hoàng", "Vũ", "Đặng", "Bùi", "Đỗ", "Ngô"]
dem = ["Văn", "Thị", "Hồng", "Quốc", "Minh", "Thành", "Thu", "Ngọc", "Đức", "Thanh", "Thiên"]
ten = ["Anh", "Bình", "Cường", "Dũng", "Hà", "Hùng", "Lan", "Mai", "Nam", "Trang", "Phong", "Duyên", "Kim"]

def gen_name():
    return f"{random.choice(ho)} {random.choice(dem)} {random.choice(ten)}"

user_df = pd.DataFrame({
    "user_id": user_id,
    "name": [gen_name() for _ in range(len(user_id))]
})

In [ ]:
user_df.to_parquet("users.parquet", engine="pyarrow", index=False)

In [31]:
movie_df = pd.read_parquet("/content/drive/MyDrive/Azure/prepared_data/movies.parquet")
user_movies = pd.read_parquet("/content/drive/MyDrive/Azure/prepared_data/user_movies.parquet")

In [32]:
user_movies = user_movies[user_movies["movie_id"].isin(movie_df["movie_id"])]

In [33]:
user_movies = user_movies.groupby("user_id")["movie_id"].apply(list).reset_index()

In [34]:
user_movies = user_movies[user_movies["movie_id"].apply(len) >= 12]

In [47]:
user_movies = user_movies.reset_index(drop = True)

In [48]:
train_df = pd.DataFrame({
    'user_id': user_movies['user_id'],
    'feature': user_movies['movie_id'].apply(lambda x: x[:-2])
})

dev_df = pd.DataFrame({
    'user_id': user_movies['user_id'],
    'dev_label': user_movies['movie_id'].apply(lambda x: x[-2:-1])
})

test_df = pd.DataFrame({
    'user_id': user_movies['user_id'],
    'test_label': user_movies['movie_id'].apply(lambda x: x[-1:])
})

train_df.to_parquet("train.parquet", engine="pyarrow", index=False)
dev_df.to_parquet("dev.parquet", engine="pyarrow", index=False)
test_df.to_parquet("test.parquet", engine="pyarrow", index=False)

In [50]:
all_df = pd.DataFrame({
    'user_id': user_movies['user_id'],
    'feature': user_movies['movie_id']
})

all_df.to_parquet("all.parquet", engine="pyarrow", index=False)